In [5]:

import pandas as pd
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
from time import sleep

# Path to the CSV file
file_path = r'C:\Users\omniq\Documents\GitHub\TIL6022-LabAssignments\Flytax-Project-Group24\CO2-effect\Data\Flight Netherlands\avia_par_nl__custom_13280147_spreadsheet.csv'

# Load the CSV file into a DataFrame using ';' as the separator
df = pd.read_csv(file_path, sep=';')

# Extract the first column and rename it to "Flight Route"
first_column = df.iloc[:, 0].rename("Flight Route")

# Extract the columns for the years 2010 to 2023
passenger_columns = df.iloc[:, 1:15].copy()  # Assuming years are in columns 2 to 15 (1:15 in 0-based indexing)
passenger_columns.columns = [str(year) for year in range(2010, 2024)]  # Setting column names for the years 2010-2023

# Create a new DataFrame with the desired columns (Flight Route and Passenger data per year)
new_df = pd.DataFrame({
    'Flight Route': first_column
})

# Append passenger data for years 2010 to 2023
new_df = pd.concat([new_df, passenger_columns], axis=1)

# Clean the 'Flight Route' column by removing unwanted prefixes and formatting the text
new_df['Flight Route'] = new_df['Flight Route'].astype(str)
new_df['Flight Route'] = new_df['Flight Route'].str.replace('AMSTERDAM/SCHIPHOL airport - ', '', regex=False)
new_df['Flight Route'] = new_df['Flight Route'].str.replace('Maastricht/Aachen airport - ', '', regex=False)
new_df['Flight Route'] = new_df['Flight Route'].str.replace('Eindhoven airport - ', '', regex=False)
new_df['Flight Route'] = new_df['Flight Route'].str.replace('Rotterdam airport - ', '', regex=False)
new_df['Flight Route'] = new_df['Flight Route'].str.replace('Groningen Eelde airport - ', '', regex=False)
new_df['Flight Route'] = new_df['Flight Route'].str.title().str.strip()

# Handle non-numeric values and clean up the passenger columns (from 2010 to 2023)
for year in range(2010, 2024):
    new_df[str(year)] = new_df[str(year)].replace(',', '', regex=True)
    new_df[str(year)] = pd.to_numeric(new_df[str(year)], errors='coerce')

# Initialize the geolocator with a user agent to avoid request issues
geolocator = Nominatim(user_agent="airport_geolocator")

# Function to get the coordinates of the airport
def get_airport_coordinates(airport_name):
    try:
        location = geolocator.geocode(airport_name)
        if location:
            return (location.latitude, location.longitude)
        else:
            retry_query = f"{airport_name} Airport"
            location = geolocator.geocode(retry_query)
            if location:
                return (location.latitude, location.longitude)
            else:
                return (None, None)
    except:
        return (None, None)

# Add a delay between requests to avoid overwhelming the geocoding service
def get_coordinates_with_delay(airport_name):
    coords = get_airport_coordinates(airport_name)
    sleep(0.05)
    return coords

# Apply the function to the 'Flight Route' column to get coordinates
new_df['Coordinates'] = new_df['Flight Route'].apply(get_coordinates_with_delay)

# Split coordinates into separate latitude and longitude columns
new_df[['Latitude', 'Longitude']] = pd.DataFrame(new_df['Coordinates'].tolist(), index=new_df.index)

# Drop the original 'Coordinates' column for clarity
new_df = new_df.drop(columns=['Coordinates'])

# Define the coordinates for the most central point in the Netherlands (Amersfoort)
central_netherlands_coords = (52.1561, 5.3878)

# Function to calculate the distance from the central point in the Netherlands to each airport
def calculate_distance_to_netherlands(lat, lon):
    if pd.notna(lat) and pd.notna(lon):
        return geodesic(central_netherlands_coords, (lat, lon)).kilometers
    else:
        return None

# Calculate the distance to the Netherlands for each airport
new_df['Distance to Netherlands (km)'] = new_df.apply(
    lambda row: calculate_distance_to_netherlands(row['Latitude'], row['Longitude']), axis=1
)

# Dictionary of known distances to Amersfoort, Netherlands for specific airports
known_distances = {
    "Montreal/Pierre Elliot Trudeau Intl, Qc Airport": 5560,
    "Toronto/Lester B. Pearson Intl, On Airport": 6000,
    "Hannover Uir Airport": 300,
    "Zagreb/Franjo Tudjman Airport": 1100,
    "Bangalore International Airport, Devenahalli, Bangalore Airport": 7800,
    "Tehran/Imam Khomaini Intl Airport": 4400,
    "Nairobi Acc,Fic,Rcc,Com Airport": 6500,
    "Luftfartstilsynet Civil Aviation Authority Airport": 1100,
    "Enfidha Zine El Abidine Ben Ali Airport": 1900,
    "Taibei City/Taibei Intl Ap Airport": 9500,
    "Kilimanjaro App, Twr, Ais, Met, Civil Airlines Airport": 7000,
    "Boston/General Edward Lawrence Logan International, Ma. Airport": 5600,
    "Washington Dulles International, Dc. Airport": 6200,
    "Maiquetia, Intl, Simon Bolivar, Vargas Airport": 7600,
    "Maastricht/Aachen Airport - Kos/Ippokratis Airport": 2100,
    "Maastricht/Aachen Airport - Zakinthos/Dionisios Solomos Airport": 2300,
    "Eindhoven Airport - Athinai/Eleftherios Venizelos Airport": 2200,
    "Eindhoven Airport - Kos/Ippokratis Airport": 2100,
    "Eindhoven Airport - Zagreb/Franjo Tudjman Airport": 1100,
    "Eindhoven Airport - Oujda/Angads Airport": 1900,
    "Eindhoven Airport - Nador/El Aroui Airport": 1900,
    "Groningen/Eelde Airport - Kos/Ippokratis Airport": 2100,
    "Rotterdam Airport - Kos/Ippokratis Airport": 2100,
    "Rotterdam Airport - Bergerac-Roumaniere Airport": 800,
    "Rotterdam Airport - Nador/El Aroui Airport": 1900,
}

# Function to update the distance in the DataFrame if it is NaN and has a known value in the dictionary
def update_distance(row):
    if pd.isna(row['Distance to Netherlands (km)']) and row['Flight Route'] in known_distances:
        return known_distances[row['Flight Route']]
    else:
        return row['Distance to Netherlands (km)']

# Apply the update function to the DataFrame to fill in the missing distances
new_df['Distance to Netherlands (km)'] = new_df.apply(update_distance, axis=1)

# Remove duplicate rows from the DataFrame
new_df = new_df.drop_duplicates()

# Display the ultimate DataFrame with all relevant columns, including NaNs where applicable
print(new_df[['Flight Route', 'Latitude', 'Longitude', 'Distance to Netherlands (km)']].to_string(index=False))

print(f"Number of rows after processing: {len(new_df)}")

# Display the passenger data for each year
print(new_df[['Flight Route'] + [str(year) for year in range(2010, 2024)]].to_string(index=False))



                                                           Flight Route   Latitude   Longitude  Distance to Netherlands (km)
                                                               Dataset:        NaN         NaN                           NaN
                                                          Last Updated:        NaN         NaN                           NaN
                                                                    Nan  46.314475   11.048029                    768.876795
                                                         Time Frequency        NaN         NaN                           NaN
                                                        Unit Of Measure        NaN         NaN                           NaN
                                      Traffic And Transport Measurement        NaN         NaN                           NaN
                                                                   Time  39.561154  -90.723413                   7034.027206
